# Enade DataSet

## Transform
1. Steps:
    * Data Discovery
    * Transform CATEGAD into "publicas" and "privadas"
    * Transform MODALIDADE into 0: EAD, 1: Presencial
    * Transform REGIAO_CURSO into 1:NO, 2:NE, 3:SE, 4:S, 5:CO

In [3]:
import pandas as pd
import numpy as np
import os

path: str = "../data/enade"
file_path: str = path + '/microdados_enade_2019/2019/3.DADOS/microdados_enade_2019.txt'

In [4]:
df = pd.read_csv(file_path, sep=';', decimal=',')
df.head()

,NU_ANO,CO_IES,CO_CATEGAD,CO_ORGACAD,CO_GRUPO,CO_CURSO,CO_MODALIDADE,CO_MUNIC_CURSO,CO_UF_CURSO,CO_REGIAO_CURSO,...,QE_I59,QE_I60,QE_I61,QE_I62,QE_I63,QE_I64,QE_I65,QE_I66,QE_I67,QE_I68
0,2019,1,10002,10028,5710,3,1,5103403,51,5,...,2.0,5.0,1.0,1.0,2.0,5.0,8.0,7.0,1.0,2.0
1,2019,1,10002,10028,5710,3,1,5103403,51,5,...,1.0,4.0,2.0,2.0,2.0,5.0,4.0,4.0,2.0,2.0
2,2019,1,10002,10028,5710,3,1,5103403,51,5,...,3.0,4.0,4.0,3.0,3.0,4.0,1.0,1.0,1.0,4.0
3,2019,1,10002,10028,5710,3,1,5103403,51,5,...,3.0,5.0,2.0,2.0,2.0,3.0,3.0,4.0,3.0,3.0
4,2019,1,10002,10028,5710,3,1,5103403,51,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
print('Total of Zero grade: {}'.format(len(df[df.NT_GER == 0])))

Total of Zero grade: 361


In [22]:
null_perc: float = df.NT_GER.isnull().sum() / df.shape[0]

print('Percentage of null: {0:.1f}%'.format(null_perc * 100))
df.NT_GER.describe()

Percentage of null: 10.1%


count    390091.000000
mean         44.076610
std          14.542059
min           0.000000
25%          33.200000
50%          43.800000
75%          54.800000
max          93.000000
Name: NT_GER, dtype: float64

In [27]:
# Região centro oeste

nulls = df.loc[df.CO_REGIAO_CURSO == 5].NT_GER.isnull().sum()
null_perc: float = nulls / df.shape[0]

print('Percentage of null: {0:.1f}%'.format(null_perc * 100))

df.loc[df.CO_REGIAO_CURSO == 5].NT_GER.describe()

Percentage of null: 0.8%


count    30769.000000
mean        44.604609
std         14.489081
min          0.000000
25%         33.900000
50%         44.500000
75%         55.300000
max         91.600000
Name: NT_GER, dtype: float64

* 01: Estado Civil
* 02: Raça
* 04: Escolaridade pai
* 05: Escolaridade mãe
* 08: Renda
* 10: Trabalho
* 11: Bolsa
* 14: Intercambio
* 15: Cotas
* 23: Horas de estudo / semana
* 25: Escolha do curso
* 26: Escolha da instituição

In [33]:
cols_of_interest = ['CO_IES', 'CO_CATEGAD', 'CO_GRUPO', 'CO_MODALIDADE', 'CO_UF_CURSO',
                    'CO_REGIAO_CURSO', 'NU_IDADE', 'TP_SEXO', 'NT_GER', 'NT_FG', 'NT_CE',
                    'QE_I01', 'QE_I02', 'QE_I04', 'QE_I05', 'QE_I08', 'QE_I10', 'QE_I11',
                    'QE_I14', 'QE_I15', 'QE_I23', 'QE_I25', 'QE_I26']
filtered_df = df[cols_of_interest].copy()

In [34]:
filtered_df.head()

,CO_IES,CO_CATEGAD,CO_GRUPO,CO_MODALIDADE,CO_UF_CURSO,CO_REGIAO_CURSO,NU_IDADE,TP_SEXO,NT_GER,NT_FG,...,QE_I04,QE_I05,QE_I08,QE_I10,QE_I11,QE_I14,QE_I15,QE_I23,QE_I25,QE_I26
0,1,10002,5710,1,51,5,27,M,51.9,36.5,...,D,E,B,A,A,A,A,D,E,C
1,1,10002,5710,1,51,5,26,M,54.2,44.3,...,E,D,E,A,A,A,A,D,E,C
2,1,10002,5710,1,51,5,23,M,NaN,NaN,...,E,E,E,B,A,A,D,B,B,A
3,1,10002,5710,1,51,5,24,M,42.1,49.9,...,F,F,E,A,A,A,A,D,C,F
4,1,10002,5710,1,51,5,41,M,NaN,NaN,...,,,,,,,,,,


## Transformations

In [35]:
filtered_df.CO_CATEGAD.unique()

array([10002,   115, 10007, 10008,    93,   121, 10001, 10003,   120,
       10005,   118, 10009,   116, 10006, 17634], dtype=int64)

In [37]:
privado = [118, 120, 121, 10005, 10006, 10007, 10008, 10009, 17634]

filtered_df.loc[
    filtered_df.CO_CATEGAD.isin(privado),
    'DESC_PUBLICA'
] = 'Privado'
filtered_df.DESC_PUBLICA.fillna('Publico', inplace=True)

In [38]:
filtered_df.DESC_PUBLICA.value_counts()

Privado    332879
Publico    101051
Name: DESC_PUBLICA, dtype: int64

In [40]:
filtered_df.CO_MODALIDADE.replace({
    0: 'EAD',
    1: 'Presencial'
}, inplace=True)

In [41]:
filtered_df.CO_MODALIDADE.value_counts()

Presencial    410542
EAD            23388
Name: CO_MODALIDADE, dtype: int64

In [42]:
filtered_df.CO_REGIAO_CURSO.replace({
    1: 'Norte',
    2: 'Nordeste',
    3: 'Sudeste',
    4: 'Sul',
    5: 'Centro Oeste'
}, inplace=True)

In [43]:
filtered_df.CO_REGIAO_CURSO.value_counts()

Sudeste         202505
Nordeste         91742
Sul              76788
Centro Oeste     34192
Norte            28703
Name: CO_REGIAO_CURSO, dtype: int64

In [51]:
filtered_df.QE_I01.replace({
    'A': 'Branca',
    'B': 'Preta',
    'C': 'Amarela',
    'D': 'Parda',
    'E': 'Indigena',
    'F': np.nan,
    ' ': np.nan
}, inplace=True)

null_perc: float = filtered_df.QE_I01.isnull().sum() / filtered_df.shape[0]
print('Percentage of null: {0:.1f}%'.format(null_perc * 100))

filtered_df.QE_I01.value_counts()

Percentage of null: 6.0%


Branca      325716
Preta        62814
Indigena     10250
Amarela       8424
Parda          671
Name: QE_I01, dtype: int64